# Web Scrape

### Import Libraby & Init url

In [4]:
import requests
import urllib.request
import time
import pandas as pd
from bs4 import BeautifulSoup

# display progress bar (tqdm>=4.23.4 | pandas==0.24.0)
from tqdm import tqdm_notebook as tqdm 

In [97]:
# init request
baseUrl = "http://www.it.kmitl.ac.th/~teerapong/news_archive"
homeUrl = "http://www.it.kmitl.ac.th/~teerapong/news_archive/index.html"
response = requests.get(homeUrl)

### Extract Month URL

In [114]:
# parse html
soup = BeautifulSoup(response.text, "html.parser", from_encoding="utf-8")

In [115]:
# create month url
li_group = soup.findAll('li')
month_url_group = [f"{baseUrl}/{li.find('a')['href']}" for li in li_group]

### Extract Article Properties 

In [135]:
# use url each month to fetch article
article_category = []
article_title = []
article_url = []

# loop each month
for month in month_url_group:
    
    # init soup
    month_resp = requests.get(month);
    soup = BeautifulSoup(month_resp.text, "html.parser", from_encoding="utf-8")
    
    # append category
    category_group = soup.findAll('td', {'class': 'category'});
    for category in category_group:
        # article category not available -> skip
        if category.getText().strip() == "N/A": continue
        article_category.append(category.getText().strip())
        
    # append title & url
    title_group = soup.findAll('td', {'class': 'title'});
    for title in title_group:
        # article title not available -> skip
        if title.getText().strip() == "Article no longer available in archive": continue
        article_title.append(title.getText().strip())
        article_url.append(f"{baseUrl}/{title.find('a')['href']}")

        
# display article properties length (check length is match)
print("Category: " + str(len(article_category)))
print("Title: " + str(len(article_title)))
print("Url: " + str(len(article_url)))

Category: 1408
Title: 1408
Url: 1408


### Extract Article Content

In [134]:
# use articule url to fetch article content
article_content = []

# init progress bar
with tqdm(total=len(my_list)) as pbar:
    
    # loop each article
    for article in article_url[:100]:

        article_resp = requests.get(article);
        soup = BeautifulSoup(article_resp.text, "html.parser", from_encoding="utf-8")
        
        current_content = []
        
        article_group = soup.findAll('p')
        for content in article_group[:-1]:

            # check <p> is empty?
            if (content.text == ""): continue
            current_content.append(content.text.rstrip("\n\r"))

        # join each <p> to raw string and append to article_content
        article_content.append(''.join(current_content))
        
        # update progress bar
        pbar.update(1)

# display article content length
print("Content: " + str(len(article_content)))


Content: 100


### Write Files

In [140]:
# write article_title.txt
with open("./datastore/article_title.txt", "w", encoding="utf-8") as file:
    for row in article_title:
        file.write("%s\n" % row)
    file.close()
    
# write article_content.txt
with open("./datastore/article_content.txt", "w", encoding="utf-8") as file:
    for row in article_content:
        file.write("%s\n" % row)
    file.close()

### Read File